# **MIP1_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """You are a city planner, looking to open facilities at some locations. We have a set of customers and a set of possible locations for opening facilities. Each potential location for establishing a facility incurs a fixed annual activation cost, which must be paid if the facility is used, regardless of the service volume it handles. Furthermore, this service volume at each facility is capped at a maximum annual limit. Additionally, there are transportation costs associated with servicing each customer from each facility.
The goal is to minimize the overall costs, which include both the fixed activation costs for any opened facilities and the transportation costs for servicing customers. This must be done while making sure that each customer’s demand is met, each facility does not exceed its maximum service volume, and customers can of course only be serviced by a facility that is opened.
Please formulate this as a mathematical optimization model."""

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [6]:
print(response1.choices[0].message.content)

Variables:

1. Binary Variable: X[j], for j in Facilities. X[j] = 1 if the facility at location j is opened, and 0 otherwise.

2. Binary Variable: Y[i][j], for i in Customers and j in Facilities. Y[i][j] = 1 if the customer i is served by facility j, and 0 otherwise.

3. Continuous Variable: S[i][j], for i in Customers and j in Facilities. S[i][j] represents the service volume provided to customer i from facility j.

4. Fixed Constant: C[j], for j in Facilities. C[j] represents the fixed annual activation cost for each facility.

5. Continuous Constant: T[i][j], for i in Customers and j in Facilities. T[i][j] represents the transportation cost of servicing customer i from facility j.

6. Continuous Constant: D[i], for i in Customers. D[i] represents the demand of each customer.

7. Continuous Constant: V[j], for j in Facilities. V[j] represents the maximum annual service volume for each facility.


In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [8]:
print(response2.choices[0].message.content)

Objective Function:

Minimize ∑_{j in Facilities}(C[j]*X[j]) + ∑_{i in Customers}∑_{j in Facilities}(T[i][j]*S[i][j])


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [11]:
print(response3.choices[0].message.content)

Constraints:

1. We must meet every customer's demand:
For each customer i in Customers, ∑_{j in Facilities}(S[i][j]) = D[i]

2. The annual service volume should not exceed the maximum limit of the facility:
For every j in Facilities, ∑_{i in Customers}(S[i][j]) <= V[j]*X[j]

3. Customers can only be served by an opened facility:
For every customer i in Customers and for every facility j in Facilities, Y[i][j] <= X[j]

4. Service volume is provided to customer i from facility j only if the customer i is being served by facility j:
For every customer i in Customers and for every facility j in Facilities, S[i][j] <= D[i]*Y[i][j]

5. All variables are non-negative (this is implicit in the definition of the variables). 

6. X[j] and Y[i][j] are binary variables. S[i][j] is a continuous variable.


### **Generate Pyomo Code**

In [12]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [13]:
Markdown(response4.choices[0].message.content)

Here is the code implemented following the above problem. You need to install the Pyomo and glpk packages to be able to run this code. 

In this example, you are going to need the glpk solver to solve the model. For the sample data, we have 2 facilities and 3 customers.

```python
from pyomo.environ import *

# Create model
model = ConcreteModel()

# Constants
facilities = 2
customers = 3
C = [10, 20]  # sample cost
T = [[1, 2], [1.5, 1], [2, 2]]  # sample transportation cost
D = [10, 15, 15]  # sample demand
V = [100, 200]  # sample service volume limit
model.F = RangeSet(facilities)
model.C = RangeSet(customers)

# Variables
model.X = Var(model.F, within=Binary)
model.Y = Var(model.C, model.F, within=Binary)
model.S = Var(model.C, model.F, within=NonNegativeReals)

# Objective
model.obj = Objective(expr=sum(C[j-1] * model.X[j] for j in model.F) +
                      sum(T[i-1][j-1] * model.S[i, j] for i in model.C for j in model.F), sense=minimize)

# Constraints
model.demandConstraint = ConstraintList()
for i in model.C:
    model.demandConstraint.add(sum(model.S[i, j] for j in model.F) == D[i-1])
    
model.volumeConstraint = ConstraintList()
for j in model.F:
    model.volumeConstraint.add(sum(model.S[i, j] for i in model.C) <= V[j-1] * model.X[j])
    
model.serveConstraint = ConstraintList()
for i in model.C:
    for j in model.F:
        model.serveConstraint.add(model.Y[i, j] <= model.X[j])
        
model.volumeServeConstraint = ConstraintList()
for i in model.C:
    for j in model.F:
        model.volumeServeConstraint.add(model.S[i, j] <= D[i-1] * model.Y[i, j])

# Solve
SolverFactory('glpk').solve(model)

# Print
for j in model.F:
    print('Facility {}:'.format(j), model.X[j].value)
for i in model.C:
    for j in model.F:
        print('Customer {}, Facility {}:'.format(i, j), model.S[i, j].value)
```
This code calculates whether it's preferable to open each facility based on a set operational cost, the demand from each customer, the cost to deliver to each customer, and the maximum delivery volume of each facility. Then it calculates the optimal amount of deliveries to send to each customer from each facility.

### **Run the code Generated by GPT4**

In [14]:
from pyomo.environ import *

# Create model
model = ConcreteModel()

# Constants
facilities = 2
customers = 3
C = [10, 20]  # sample cost
T = [[1, 2], [1.5, 1], [2, 2]]  # sample transportation cost
D = [10, 15, 15]  # sample demand
V = [100, 200]  # sample service volume limit
model.F = RangeSet(facilities)
model.C = RangeSet(customers)

# Variables
model.X = Var(model.F, within=Binary)
model.Y = Var(model.C, model.F, within=Binary)
model.S = Var(model.C, model.F, within=NonNegativeReals)

# Objective
model.obj = Objective(expr=sum(C[j-1] * model.X[j] for j in model.F) +
                      sum(T[i-1][j-1] * model.S[i, j] for i in model.C for j in model.F), sense=minimize)

# Constraints
model.demandConstraint = ConstraintList()
for i in model.C:
    model.demandConstraint.add(sum(model.S[i, j] for j in model.F) == D[i-1])

model.volumeConstraint = ConstraintList()
for j in model.F:
    model.volumeConstraint.add(sum(model.S[i, j] for i in model.C) <= V[j-1] * model.X[j])

model.serveConstraint = ConstraintList()
for i in model.C:
    for j in model.F:
        model.serveConstraint.add(model.Y[i, j] <= model.X[j])

model.volumeServeConstraint = ConstraintList()
for i in model.C:
    for j in model.F:
        model.volumeServeConstraint.add(model.S[i, j] <= D[i-1] * model.Y[i, j])

# Solve
SolverFactory('glpk').solve(model)

# Print
for j in model.F:
    print('Facility {}:'.format(j), model.X[j].value)
for i in model.C:
    for j in model.F:
        print('Customer {}, Facility {}:'.format(i, j), model.S[i, j].value)

Facility 1: 1.0
Facility 2: 0.0
Customer 1, Facility 1: 10.0
Customer 1, Facility 2: 0.0
Customer 2, Facility 1: 15.0
Customer 2, Facility 2: 0.0
Customer 3, Facility 1: 15.0
Customer 3, Facility 2: 0.0


In [35]:


from pyomo.environ import *

# Create model
model = ConcreteModel()

# Constants
facilities = 3
customers = 5
C = [1000,1000,1000]  # sample cost
T = {
    (1, 1): 4,  (1, 2): 6,  (1, 3): 9,
    (2, 1): 5,  (2, 2): 4,  (2, 3): 7,
    (3, 1): 6,  (3, 2): 3,  (3, 3): 4,
    (4, 1): 8,  (4, 2): 5,  (4, 3): 3,
    (5, 1): 10, (5, 2): 8,  (5, 3): 4,
}  # sample transportation cost

D = [80,270,250,160,180]  # sample demand
V = [500,500,500]  # sample service volume limit
model.F = RangeSet(facilities)
model.C = RangeSet(customers)

# Variables
model.X = Var(model.F, within=Binary)
model.Y = Var(model.C, model.F, within=Binary)
model.S = Var(model.C, model.F, within=NonNegativeReals)

# Objective
model.obj = Objective(expr=sum(C[j-1] * model.X[j] for j in model.F) +
                      sum(T[i,j] * model.S[i, j] for i in model.C for j in model.F), sense=minimize)

# Constraints
model.demandConstraint = ConstraintList()
for i in model.C:
    model.demandConstraint.add(sum(model.S[i, j] for j in model.F) == D[i-1])

model.volumeConstraint = ConstraintList()
for j in model.F:
    model.volumeConstraint.add(sum(model.S[i, j] for i in model.C) <= V[j-1] * model.X[j])

model.serveConstraint = ConstraintList()
for i in model.C:
    for j in model.F:
        model.serveConstraint.add(model.Y[i, j] <= model.X[j])

model.volumeServeConstraint = ConstraintList()
for i in model.C:
    for j in model.F:
        model.volumeServeConstraint.add(model.S[i, j] <= D[i-1] * model.Y[i, j])

# Solve
SolverFactory('glpk').solve(model)

# Print
for j in model.F:
    print('Facility {}:'.format(j), model.X[j].value)
for i in model.C:
    for j in model.F:
        print('Customer {}, Facility {}:'.format(i, j), model.S[i, j].value)
print(model.obj())

Facility 1: 0.0
Facility 2: 1.0
Facility 3: 1.0
Customer 1, Facility 1: 5.91171556152404e-14
Customer 1, Facility 2: 80.0
Customer 1, Facility 3: 0.0
Customer 2, Facility 1: -4.27677702188687e-13
Customer 2, Facility 2: 270.0
Customer 2, Facility 3: -1.4031077886857e-14
Customer 3, Facility 1: 0.0
Customer 3, Facility 2: 150.0
Customer 3, Facility 3: 99.9999999999999
Customer 4, Facility 1: 0.0
Customer 4, Facility 2: 0.0
Customer 4, Facility 3: 160.0
Customer 5, Facility 1: 0.0
Customer 5, Facility 2: 0.0
Customer 5, Facility 3: 180.0
5609.999999999997


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**